# Ecommerce Consumer Behavior Analysis  
## 02 – Data Cleaning & Feature Engineering


In [2]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")


In [6]:
file_path = "../data/raw/ecommerce_consumer_behavior.csv"
df = pd.read_csv(file_path)

df.shape
df.head()


,Customer_ID,Age,Gender,Income_Level,Marital_Status,Education_Level,Occupation,Location,Purchase_Category,Purchase_Amount,Frequency_of_Purchase,Purchase_Channel,Brand_Loyalty,Product_Rating,Time_Spent_on_Product_Research(hours),Social_Media_Influence,Discount_Sensitivity,Return_Rate,Customer_Satisfaction,Engagement_with_Ads,Device_Used_for_Shopping,Payment_Method,Time_of_Purchase,Discount_Used,Customer_Loyalty_Program_Member,Purchase_Intent,Shipping_Preference,Time_to_Decision
0,37-611-6911,22,Female,Middle,Married,Bachelor's,Middle,Évry,Gardening & Outdoors,$333.80,4,Mixed,5,5,2.00,NaN,Somewhat Sensitive,1,7,NaN,Tablet,Credit Card,03-01-2024,True,False,Need-based,No Preference,2
1,29-392-9296,49,Male,High,Married,High School,High,Huocheng,Food & Beverages,$222.22,11,In-Store,3,1,2.00,Medium,Not Sensitive,1,5,High,Tablet,PayPal,4/16/2024,True,False,Wants-based,Standard,6
2,84-649-5117,24,Female,Middle,Single,Master's,High,Huzhen,Office Supplies,$426.22,2,Mixed,5,5,0.30,Low,Not Sensitive,1,7,Low,Smartphone,Debit Card,3/15/2024,True,True,Impulsive,No Preference,3
3,48-980-6078,29,Female,Middle,Single,Master's,Middle,Wiwilí,Home Appliances,$101.31,6,Mixed,3,1,1.00,High,Somewhat Sensitive,0,1,NaN,Smartphone,Other,10-04-2024,True,True,Need-based,Express,10
4,91-170-9072,33,Female,Middle,Widowed,High School,Middle,Nara,Furniture,$211.70,6,Mixed,3,4,0.00,Medium,Not Sensitive,2,10,NaN,Smartphone,Debit Card,1/30/2024,False,False,Wants-based,No Preference,4


In [7]:
# time column(s)
df["Time_of_Purchase"] = pd.to_datetime(df["Time_of_Purchase"], errors="coerce")

df.dtypes
df[["Time_of_Purchase"]].head()


,Time_of_Purchase
0,2024-03-01
1,NaT
2,NaT
3,2024-10-04
4,NaT


In [8]:
print("Duplicate full rows before:", df.duplicated().sum())
# df = df.drop_duplicates()

Duplicate full rows before: 0


In [9]:
df = df[~df["Customer_ID"].isna()]


In [10]:
missing = df.isna().sum()
missing[missing > 0].sort_values(ascending=False)

Time_of_Purchase          618
Engagement_with_Ads       256
Social_Media_Influence    247
dtype: int64

Missing value strategy

- Demographic fields (Age, Income_Level, Education_Level, etc.): impute with "Unknown" or median/most frequent.
- Behaviour fields (Time_Spent_on_Product_Research, Engagement_with_Ads, etc.): impute with median where reasonable, else leave as-is and flag.
- Time_of_Purchase: rows where time is completely invalid are very few, so they can be dropped.

In [11]:
cat_fill_unknown = [
    "Income_Level", "Marital_Status", "Education_Level", "Occupation",
    "Location", "Purchase_Channel", "Brand_Loyalty",
    "Social_Media_Influence", "Discount_Sensitivity",
    "Customer_Satisfaction", "Engagement_with_Ads",
    "Device_Used_for_Shopping", "Payment_Method",
    "Customer_Loyalty_Program_Member", "Purchase_Intent",
    "Shipping_Preference", "Gender"
]

for col in cat_fill_unknown:
    if col in df.columns:
        df[col] = df[col].fillna("Unknown")

In [12]:
df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

# 2) Purchase_Amount ko clean + convert
df["Purchase_Amount"] = (
    df["Purchase_Amount"]
        .astype(str)
        .str.replace(r"[\$,]", "", regex=True)  # $ and , hatao
        .str.strip()
)

df["Purchase_Amount"] = pd.to_numeric(df["Purchase_Amount"], errors="coerce")

df["Purchase_Amount"].head()
df["Purchase_Amount"].dtype


dtype('float64')

In [13]:
num_fill_median = [
    "Age",
    "Purchase_Amount",
    "Frequency_of_Purchase",
    "Product_Rating",
    "Time_Spent_on_Product_Research(hours)",
    "Return_Rate",
    "Time_to_Decision"
]

for col in num_fill_median:
    if col in df.columns:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)
# # df.info()
# df.head(20)

In [22]:
df.isna().sum().sort_values(ascending=False)

Time_of_Purchase                         618
Customer_ID                                0
Age                                        0
Shipping_Preference                        0
Purchase_Intent                            0
Customer_Loyalty_Program_Member            0
Discount_Used                              0
Payment_Method                             0
Device_Used_for_Shopping                   0
Engagement_with_Ads                        0
Customer_Satisfaction                      0
Return_Rate                                0
Discount_Sensitivity                       0
Social_Media_Influence                     0
Time_Spent_on_Product_Research(hours)      0
Product_Rating                             0
Brand_Loyalty                              0
Purchase_Channel                           0
Frequency_of_Purchase                      0
Purchase_Amount                            0
Purchase_Category                          0
Location                                   0
Occupation

In [14]:
#  Time column – drop rows where time can't be parsed
df = df[~df["Time_of_Purchase"].isna()]

In [15]:
df.isna().sum().sort_values(ascending=False)

Customer_ID                              0
Age                                      0
Shipping_Preference                      0
Purchase_Intent                          0
Customer_Loyalty_Program_Member          0
Discount_Used                            0
Time_of_Purchase                         0
Payment_Method                           0
Device_Used_for_Shopping                 0
Engagement_with_Ads                      0
Customer_Satisfaction                    0
Return_Rate                              0
Discount_Sensitivity                     0
Social_Media_Influence                   0
Time_Spent_on_Product_Research(hours)    0
Product_Rating                           0
Brand_Loyalty                            0
Purchase_Channel                         0
Frequency_of_Purchase                    0
Purchase_Amount                          0
Purchase_Category                        0
Location                                 0
Occupation                               0
Education_L

In [16]:
df[["Purchase_Amount", "Frequency_of_Purchase",
    "Return_Rate", "Product_Rating",
    "Discount_Used", "Time_Spent_on_Product_Research(hours)",
    "Time_to_Decision"]].describe().T

,count,mean,std,min,25%,50%,75%,max
Purchase_Amount,382.00,277.05,129.01,50.78,166.40,279.27,385.26,498.33
Frequency_of_Purchase,382.00,7.08,3.10,2.00,4.00,7.00,10.00,12.00
Return_Rate,382.00,0.95,0.80,0.00,0.00,1.00,2.00,2.00
Product_Rating,382.00,2.87,1.46,1.00,2.00,3.00,4.00,5.00
Time_Spent_on_Product_Research(hours),382.00,1.02,0.77,0.00,0.00,1.00,2.00,2.00
Time_to_Decision,382.00,7.55,3.95,1.00,4.00,8.00,11.00,14.00


In [17]:
(df["Purchase_Amount"] < 0).sum(), (df["Frequency_of_Purchase"] < 0).sum()

(0, 0)

In [18]:
# Rating outside expected range (assumeing 1–5)
out_rating = df[(df["Product_Rating"] < 1) | (df["Product_Rating"] > 5)]
out_rating.shape

(0, 28)

In [19]:
# Example: clip rating to 1–5
df["Product_Rating"] = df["Product_Rating"].clip(1, 5)

In [20]:
q1 = df["Purchase_Amount"].quantile(0.01)
q99 = df["Purchase_Amount"].quantile(0.99)
df["Purchase_Amount_clipped"] = df["Purchase_Amount"].clip(q1, q99)


In [21]:
df["Purchase_Date"] = df["Time_of_Purchase"].dt.date
df["Purchase_Year"] = df["Time_of_Purchase"].dt.year
df["Purchase_Month"] = df["Time_of_Purchase"].dt.month
df["Purchase_Month_Name"] = df["Time_of_Purchase"].dt.month_name()
df["Purchase_DayOfWeek"] = df["Time_of_Purchase"].dt.day_name()
df["Purchase_Hour"] = df["Time_of_Purchase"].dt.hour

In [35]:
df.head(10)

,Customer_ID,Age,Gender,Income_Level,Marital_Status,Education_Level,Occupation,Location,Purchase_Category,Purchase_Amount,Frequency_of_Purchase,Purchase_Channel,Brand_Loyalty,Product_Rating,Time_Spent_on_Product_Research(hours),Social_Media_Influence,Discount_Sensitivity,Return_Rate,Customer_Satisfaction,Engagement_with_Ads,Device_Used_for_Shopping,Payment_Method,Time_of_Purchase,Discount_Used,Customer_Loyalty_Program_Member,Purchase_Intent,Shipping_Preference,Time_to_Decision,Purchase_Amount_clipped,Purchase_Date,Purchase_Year,Purchase_Month,Purchase_Month_Name,Purchase_DayOfWeek,Purchase_Hour
0,37-611-6911,22,Female,Middle,Married,Bachelor's,Middle,Évry,Gardening & Outdoors,333.80,4,Mixed,5,5,2.00,Unknown,Somewhat Sensitive,1,7,Unknown,Tablet,Credit Card,2024-03-01,True,False,Need-based,No Preference,2,333.80,2024-03-01,2024,3,March,Friday,0
3,48-980-6078,29,Female,Middle,Single,Master's,Middle,Wiwilí,Home Appliances,101.31,6,Mixed,3,1,1.00,High,Somewhat Sensitive,0,1,Unknown,Smartphone,Other,2024-10-04,True,True,Need-based,Express,10,101.31,2024-10-04,2024,10,October,Friday,0
8,37-065-3182,24,Female,High,Divorced,Master's,Middle,Gråbo,Office Supplies,201.96,8,In-Store,3,5,0.00,Low,Very Sensitive,1,2,High,Desktop,Cash,2024-06-10,False,False,Need-based,Standard,7,201.96,2024-06-10,2024,6,June,Monday,0
9,84-894-9222,25,Bigender,High,Married,Bachelor's,High,Pryamitsyno,Sports & Outdoors,418.83,7,Online,2,5,1.00,Unknown,Somewhat Sensitive,1,5,Medium,Tablet,PayPal,2024-10-07,True,False,Need-based,No Preference,13,418.83,2024-10-07,2024,10,October,Monday,0
10,44-674-4037,33,Male,Middle,Divorced,Bachelor's,Middle,Punta Gorda,Mobile Accessories,389.70,12,In-Store,1,4,0.50,Low,Not Sensitive,0,4,High,Desktop,PayPal,2024-07-02,True,False,Planned,Express,12,389.70,2024-07-02,2024,7,July,Tuesday,0
12,19-933-8095,34,Male,Middle,Single,Bachelor's,High,Gaosheng,Luxury Goods,61.22,7,In-Store,5,2,2.00,Low,Very Sensitive,1,7,High,Desktop,Debit Card,2024-08-05,True,True,Need-based,No Preference,5,61.22,2024-08-05,2024,8,August,Monday,0
15,90-069-8934,40,Male,High,Divorced,Master's,High,Dhankutā,Hotels,109.29,3,Online,3,4,0.00,Medium,Somewhat Sensitive,2,2,High,Desktop,Debit Card,2024-08-11,False,True,Wants-based,Express,11,109.29,2024-08-11,2024,8,August,Sunday,0
17,89-883-2101,38,Female,Middle,Divorced,Bachelor's,High,Ngeru,Mobile Accessories,236.99,10,Mixed,2,4,0.00,High,Very Sensitive,0,8,Unknown,Tablet,PayPal,2024-12-09,True,True,Impulsive,No Preference,13,236.99,2024-12-09,2024,12,December,Monday,0
19,67-159-7366,29,Female,High,Widowed,Bachelor's,Middle,Monastyrshchina,Electronics,490.75,10,In-Store,2,4,1.00,Unknown,Very Sensitive,1,4,Low,Smartphone,Debit Card,2024-07-08,False,False,Wants-based,Standard,13,490.75,2024-07-08,2024,7,July,Monday,0
23,95-478-2650,37,Female,Middle,Single,Bachelor's,Middle,São Roque,Software & Apps,461.00,7,Online,2,4,1.00,High,Not Sensitive,0,8,Low,Tablet,Cash,2024-07-10,False,True,Wants-based,Standard,9,461.00,2024-07-10,2024,7,July,Wednesday,0


In [22]:
snapshot_date = df["Time_of_Purchase"].max() + pd.Timedelta(days=1)
cust_agg = df.groupby("Customer_ID").agg(
    first_purchase_date = ("Time_of_Purchase", "min"),
    last_purchase_date = ("Time_of_Purchase", "max"),
    total_purchases = ("Purchase_Amount", "count"),
    total_revenue = ("Purchase_Amount", "sum"),
    avg_purchase_value = ("Purchase_Amount", "mean"),
    total_frequency = ("Frequency_of_Purchase", "sum"),
    mean_frequency = ("Frequency_of_Purchase", "mean"),
    avg_rating = ("Product_Rating", "mean"),
    avg_discount_used = ("Discount_Used", "mean"),
    mean_return_rate = ("Return_Rate", "mean"),
)

cust_agg["recency_days"] = (snapshot_date - cust_agg["last_purchase_date"]).dt.days
cust_agg.head()


,first_purchase_date,last_purchase_date,total_purchases,total_revenue,avg_purchase_value,total_frequency,mean_frequency,avg_rating,avg_discount_used,mean_return_rate,recency_days
Customer_ID,,,,,,,,,,,
00-149-4481,2024-02-04,2024-02-04,1,464.76,464.76,2,2.00,1.00,0.00,0.00,313
00-265-0556,2024-11-01,2024-11-01,1,68.02,68.02,10,10.00,2.00,1.00,1.00,42
00-335-5034,2024-08-12,2024-08-12,1,411.40,411.40,3,3.00,3.00,1.00,2.00,123
00-520-4376,2024-10-03,2024-10-03,1,97.97,97.97,2,2.00,5.00,1.00,0.00,71
00-733-8770,2024-12-08,2024-12-08,1,57.50,57.50,12,12.00,1.00,1.00,0.00,5


In [23]:
def r_score(rec):
    if rec <= 30:
        return 4          # purchased very recently
    elif rec <= 90:
        return 3
    elif rec <= 180:
        return 2
    else:
        return 1          # long time since last purchase

cust_agg["R_score"] = cust_agg["recency_days"].apply(r_score)

# --- Frequency score (higher total_purchases = better) ---
def f_score(freq):
    if freq <= 1:
        return 1          # one‑time buyer
    elif freq <= 3:
        return 2
    elif freq <= 6:
        return 3
    else:
        return 4          # very frequent

cust_agg["F_score"] = cust_agg["total_purchases"].apply(f_score)

# --- Monetary score (higher total_revenue = better) ---
q25 = cust_agg["total_revenue"].quantile(0.25)
q50 = cust_agg["total_revenue"].quantile(0.50)
q75 = cust_agg["total_revenue"].quantile(0.75)

def m_score(m):
    if m <= q25:
        return 1
    elif m <= q50:
        return 2
    elif m <= q75:
        return 3
    else:
        return 4

cust_agg["M_score"] = cust_agg["total_revenue"].apply(m_score)

# --- Final RFM score ---
cust_agg["RFM_score"] = (
    cust_agg["R_score"] + cust_agg["F_score"] + cust_agg["M_score"]
)

In [24]:
def rfm_segment(row):
    s = row["RFM_score"]
    if s >= 10:
        return "Champions"
    elif s >= 8:
        return "Loyal"
    elif s >= 6:
        return "Potential Loyalists"
    elif s >= 5:
        return "At Risk"
    else:
        return "Low Value"

cust_agg["RFM_segment"] = cust_agg.apply(rfm_segment, axis=1)

cust_agg["RFM_segment"].value_counts()


RFM_segment
Potential Loyalists    150
Low Value              113
At Risk                 92
Loyal                   27
Name: count, dtype: int64

 Repeat vs one‑time customer

In [25]:
cust_agg["is_repeat_customer"] = np.where(cust_agg["total_purchases"] > 1, 1, 0)

Join back to main df for EDA:

In [28]:
df = df.merge(
    cust_agg[["is_repeat_customer", "RFM_segment"]],
    left_on = "Customer_ID",
    right_index = True,
    how = "left"
)

Discount sensitivity bucket

In [32]:
def map_senstivity(x):
    if isinstance(x, str):
        return x
    if x <= 0.33:
        return "Low"
    elif x <= 0.66:
        return "Medium"
    else:
        return "High"
    
if df["Discount_Sensitivity"].dtype != "object":
    df["Discount_Sensitivity_Bucket"] = df["Discount_Sensitivity"].apply(map_senstivity)
else:
    df["Discount_Sensitivity_Bucket"] = df["Discount_Sensitivity"]

In [33]:
def dicision_speed(x):
    if x <= df["Time_to_Decision"].quantile(0.25):
        return "Fast"
    elif x <= df["Time_to_Decision"].quantile(0.75):
        return "Medium"
    else:
        return "Slow"

df["Decision_Speed"] = df["Time_to_Decision"].apply(dicision_speed)

Final checks on engineered dataset

In [34]:
df.head()

,Customer_ID,Age,Gender,Income_Level,Marital_Status,Education_Level,Occupation,Location,Purchase_Category,Purchase_Amount,Frequency_of_Purchase,Purchase_Channel,Brand_Loyalty,Product_Rating,Time_Spent_on_Product_Research(hours),Social_Media_Influence,Discount_Sensitivity,Return_Rate,Customer_Satisfaction,Engagement_with_Ads,Device_Used_for_Shopping,Payment_Method,Time_of_Purchase,Discount_Used,Customer_Loyalty_Program_Member,Purchase_Intent,Shipping_Preference,Time_to_Decision,Purchase_Amount_clipped,Purchase_Date,Purchase_Year,Purchase_Month,Purchase_Month_Name,Purchase_DayOfWeek,Purchase_Hour,is_repeat_customer,RFM_segment,Discount_Sensitivity_Bucket,Decision_Speed
0,37-611-6911,22,Female,Middle,Married,Bachelor's,Middle,Évry,Gardening & Outdoors,333.80,4,Mixed,5,5,2.00,Unknown,Somewhat Sensitive,1,7,Unknown,Tablet,Credit Card,2024-03-01,True,False,Need-based,No Preference,2,333.80,2024-03-01,2024,3,March,Friday,0,0,At Risk,Somewhat Sensitive,Fast
3,48-980-6078,29,Female,Middle,Single,Master's,Middle,Wiwilí,Home Appliances,101.31,6,Mixed,3,1,1.00,High,Somewhat Sensitive,0,1,Unknown,Smartphone,Other,2024-10-04,True,True,Need-based,Express,10,101.31,2024-10-04,2024,10,October,Friday,0,0,At Risk,Somewhat Sensitive,Medium
8,37-065-3182,24,Female,High,Divorced,Master's,Middle,Gråbo,Office Supplies,201.96,8,In-Store,3,5,0.00,Low,Very Sensitive,1,2,High,Desktop,Cash,2024-06-10,False,False,Need-based,Standard,7,201.96,2024-06-10,2024,6,June,Monday,0,0,Low Value,Very Sensitive,Medium
9,84-894-9222,25,Bigender,High,Married,Bachelor's,High,Pryamitsyno,Sports & Outdoors,418.83,7,Online,2,5,1.00,Unknown,Somewhat Sensitive,1,5,Medium,Tablet,PayPal,2024-10-07,True,False,Need-based,No Preference,13,418.83,2024-10-07,2024,10,October,Monday,0,0,Loyal,Somewhat Sensitive,Slow
10,44-674-4037,33,Male,Middle,Divorced,Bachelor's,Middle,Punta Gorda,Mobile Accessories,389.70,12,In-Store,1,4,0.50,Low,Not Sensitive,0,4,High,Desktop,PayPal,2024-07-02,True,False,Planned,Express,12,389.70,2024-07-02,2024,7,July,Tuesday,0,0,Potential Loyalists,Not Sensitive,Slow


In [35]:
df.shape

(382, 39)

In [36]:
df.isna().sum().sort_values(ascending=False).head(10)


Customer_ID                        0
Purchase_Date                      0
Time_of_Purchase                   0
Discount_Used                      0
Customer_Loyalty_Program_Member    0
Purchase_Intent                    0
Shipping_Preference                0
Time_to_Decision                   0
Purchase_Amount_clipped            0
Purchase_Year                      0
dtype: int64

In [43]:
df["RFM_segment"].value_counts()
df["is_repeat_customer"].value_counts()
df["Decision_Speed"].value_counts()

RFM_segment
Potential Loyalists    150
Low Value              113
At Risk                 92
Loyal                   27
Name: count, dtype: int64

In [44]:
output_path = "../data/cleaned/ecommerce_behavior_cleaned.csv"
df.to_csv(output_path, index=False)
output_path

'../data/cleaned/ecommerce_behavior_cleaned.csv'

In [45]:
cust_output_path = "../data/cleaned/customer_master_rfm.csv"
cust_agg.to_csv(cust_output_path, index=True)  # index is Customer_ID
cust_output_path

'../data/cleaned/customer_master_rfm.csv'

Cleaning & Feature Engineering Summary

In this notebook I:

- Removed exact duplicate rows and rows with invalid `Time_of_Purchase`.
- Treated missing values:
  - Filled categorical demographics with `"Unknown"`.
  - Imputed numeric behavioural fields with median values.
- Fixed obvious data issues (e.g. clipping impossible product ratings / extreme purchase amounts).
- Engineered key features:
  - Time-based fields: year, month, weekday, hour, part of day.
  - Customer-level metrics: total_purchases, total_revenue, avg_order_value, recency_days.
  - RFM scores and segments (Champions, Loyal, Potential Loyalists, At Risk, Low Value).
  - Repeat customer flag and decision speed buckets.
- Saved:
  - A cleaned transactional dataset for EDA and visualization.
  - A customer master table for customer behaviour and retention analysis.
